In [1]:
import os
# I need to visualize all the environment varibale defined
# in the notebook. I need to know the order of the cells
# and the order of the variables defined in each cell.

os.environ.pop("OPENAI_API_BASE", None)


In [2]:
# Import some libraries and especially load environment variables
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# set an environment varibale
os.environ["LANGCHAIN_PROJECT"] = "01_basic"

print(os.getenv("AZURE_ENDPOINT"))
print(os.getenv("OPENAI_API_BASE"))
# I need to clear the OPENAI_API_BASE variable

os.environ.pop("OPENAI_API_BASE", None)
print(os.getenv("OPENAI_API_BASE"))


https://alkopenai2.openai.azure.com/
https://alkopenai2.openai.azure.com/
None


In [3]:
# Create an LLM based on GPT4 deployed on Azure
llm = AzureChatOpenAI(
    openai_api_version="2023-09-01-preview",
    deployment_name="Gpt35_2", 
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    model_name="gpt-3.5-turbo-1106"
)

In [4]:
# I the import my custom function to work with audio and videp
from plugins.AudioVideo import AudioVideo

audio_video = AudioVideo()

# you can test the function
audio_video.extract_audio("C:\\temp\\ssh.mp4")

Extracting auio file from video C:\temp\ssh.mp4


'C:\\temp\\ssh.wav'

In [5]:
from plugins.Summarization import Summarization
summarization = Summarization()

# summarization.summarize_timeline(""" 00:00	 Hi and welcome again to a new video about UBQI, in which I deal with a question I got quite often and the question is why I cannot use advanced features like
# 00:14	 SSH key stored in my UBQI when I work in Windows, it seems not supported. So what is the problem?
# 00:22	 This problem usually does not happen in Linux because Linux notively has a support for SSH so when you update your Linux machine,
# 00:31	 usually you will receive the latest version of the open SSH protocol that supports UBQI and you have not problem.
# 00:39	 In Windows, this situation is a little different. Let me show you why.
# 00:45	 In the vast majority of C2H shown, the problem is not having the latest version of SSH
# 00:50	 and to verify this open a Windows terminal and type SSH, sorry, SSH-cabbital day.
# 00:58	 And if you did not ever install or upgrade SSH, this is the answer.
# 01:06	 The answer is open SSH for Windows 8.6 and this is indeed too old to support your UBQI.
# """)

In [6]:
# Then create tools list
from langchain.tools import BaseTool, StructuredTool, Tool, tool

tools = [
    Tool.from_function(
        func=audio_video.extract_audio,
        name="ExtractAudio",
        description="extract audio in wav format from an mp4 file",
    ),
    Tool.from_function(
        func=audio_video.transcript_timeline,
        name="TranscriptTimeline",
        description="Transcript audio from a wav file to a timeline",
    ),
    Tool.from_function(
        func=summarization.summarize_timeline,
        name="SummarizeTimeline",
        description="Take a transcribed timeline and create a summarized timeline.",
    ),
]

In [7]:
# Finally you can create the agent
from langchain.agents import AgentType, initialize_agent
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
# Everthing is ready to start the conversation
agent.run(
    "I need a summarized timeline from video /Users/gianmariaricci/develop/montaggi/RDP.mp4"
)

In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
prompt = hub.pull("hwchase17/openai-functions-agent")

agentFunction =  create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agentFunction, tools=tools, verbose=True)

agent_executor({
    "input": "I need a summarized timeline from video /Users/gianmariaricci/develop/montaggi/RDP.mp4"})






> Entering new AgentExecutor chain...

Invoking: `ExtractAudio` with `/Users/gianmariaricci/develop/montaggi/RDP.mp4`


Extracting auio file from video /Users/gianmariaricci/develop/montaggi/RDP.mp4
/Users/gianmariaricci/develop/montaggi/RDP.wav
Invoking: `TranscriptTimeline` with `/Users/gianmariaricci/develop/montaggi/RDP.wav`


Extracting transcript from audio file /Users/gianmariaricci/develop/montaggi/RDP.wav


/Users/gianmariaricci/develop/github/LangChainPlayground/lchain/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 